In [ ]:
# Base modules

from numpy import load
import numpy as np
from pathlib import Path

# Import os & glob
import os
import glob

# Import plot tools
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import matplotlib.axes as axes
from mpl_toolkits import mplot3d

# Import data analysis tools
from scipy.signal import savgol_filter
from scipy import interpolate
import pandas as pd
from scipy.signal import find_peaks

import seaborn as sns
import matplotlib.gridspec as grid_spec

from statsmodels.graphics import tsaplots
import random

# Import timing
import time
import natsort
import itertools

from sklearn.utils import shuffle

import plotly.express as px
from tqdm.notebook import tqdm

%matplotlib widget

In [ ]:
folders_path = glob.glob("C:/PhD/17010223/long-term-free-swim/t1/*.csv.gz")

In [ ]:
#define functions for processing

def moving_avg_interpolate(df, window_size=11, interpolate_method='pad'):
    """
    Runs across the dataset- smoothens it and then interpolates;
    followed by finding difference between two consecutive datapoints
    Args:
        df (dataframe): dataframe containing the data
        window_size (int): window size for moving average
        interpolate_method (str): method for interpolation
    Returns:
        2-tuple: (dataframe, dataframe)
    """
    sdf= df.rolling(window_size,center=True).mean().interpolate(method=interpolate_method)
    ddf=sdf.diff().interpolate(method='bfill')
    return sdf, ddf

In [ ]:
def calc_params(df,):
    """
    Reads the data from the file and processes it.
    """
    global speed, angle, peaks, angle_unwrap, angle_burst, dangle, dangle_centered, time_burst, exp_time
    # Read the data from the file
    df['zs'], df['dz'] = moving_avg_interpolate(df['fishz'])
    df['ys'], df['dy'] = moving_avg_interpolate(df['fishy'])
    df['xs'], df['dx'] = moving_avg_interpolate(df['fishx'])

    angle = np.arctan2(df['fishy'], df['fishx'])
    df['angle'] = angle
    print('Angle is', angle)

    # exp_time = (df["realtime"] - df["realtime"].iloc[0])/60
    # df["exp_time"] = exp_time

    exp_time = (df["start_time"] - df["start_time"].iloc[0])/60
    df["exp_time"] = exp_time
    
    # calculate speed
    fHz= 100 #fps
    dt = 1/fHz

    speed = np.sqrt(df['fishx']**2 + df['fishy']**2 + df['fishz']**2)/dt
    df['speed'] = speed
    print('Speed is', speed)
    
    ang_int = []
    tmp = 0
    
    for ii in range(int(1*(len(angle)-1))):
        tmp += angle[ii]
        ang_int.append(tmp)

    angle_unwrap = np.array(ang_int)
        
    frames_btw_2bouts = round(fHz/5)
    bout_width = round(fHz/100)

    h = [0.02,0.3]

    # gets the indices at which if finds peaks
    peaks,_ = find_peaks(speed, height = h, distance = frames_btw_2bouts, prominence = 0.05)

    # get the values of phi only at burst events (discretization)

    angle_burst = angle[peaks]
    dangle = angle_burst.diff()#.abs()%360
    dangle_centered = ((dangle + 2*np.pi)% 2*np.pi)- np.pi

    #  get the time of the burst events
    time_burst = df.exp_time[peaks]

    return speed, angle, peaks, angle_unwrap, angle_burst, dangle, dangle_centered, exp_time, time_burst, Mean_sq_displacement